とりあえずjndiという文字列が入ったものを全期間にて取り出しを行った後、
JNDILookUpに該当する文字列、具体的には

- $jndi
- ${jndi
- {jndi
- %25%7Bjndi
- %25jndi
- %7Bjndi
とあるものを取り出す。

In [3]:
import json
from ElasticSearcher import search
from Grapher import visualizeTimeData

query = {
    "wildcard": {
        "request": "*jndi*"
    }
}
r = search(query=query, starting=1911, ending=2112)
with open('resultData/hit_JNDI.json', 'w') as f:
    json.dump(r, f, indent=4)

c:\users\shion\pycharmprojects\projectroute112021\venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


以下の調査で判明したこと
- 当該ハニーポットでは、CVE-2021-44228は2021/12/9以前は検出されなかった。
-

In [2]:
r1 = []
hits = ['{jndi', '%25jndi', "%7bjndi"]
for d in r:
    x = d['_source']['request'].lower()
    for h in hits:
        if h in x:
            r1.append(d)
            break
print(len(r1))
print(len(r))
import pandas as pd

t = pd.to_datetime("2021/12/09 12:00:00").timestamp()
b = False
for d in r1:
    p_time = pd.to_datetime(d['_source']['@timestamp']).timestamp()
    if p_time < t:
        b = True
        break
if not b: print("no CVE-2021-44228 found before 2021/12/9")
ending = pd.to_datetime("2021/12/21 00:00:00")
visualizeTimeData(r1, starting=t, ending=ending.timestamp(), title='CVE-2021-44228 全データの期間分布',
                  loc="Log4J調査", period=12 * 60 * 60, x_axis='hour')
starting = pd.to_datetime("2021/12/09 12:00:00")
ending = pd.to_datetime("2021/12/10 00:00:00")
visualizeTimeData(r1, starting=t, ending=ending.timestamp(), title='CVE-2021-44228 初期動向',
                  loc="Log4J調査", period=1 * 30 * 60, x_axis='hour')
initialData = []
ending = pd.to_datetime("2021/12/10 12:00:00")
for d in r1:
    p_time = pd.to_datetime(d['_source']['@timestamp']).timestamp()
    if p_time < ending.timestamp():
        print(d['_source']['source_ip'])
        print(d['_source']['geoip']['city_name'] if 'city_name' in d['_source']['geoip'] else '')

9699
11467
no CVE-2021-44228 found before 2021/12/9
107.189.1.160
Roost
171.25.193.25

185.220.101.147

194.48.199.78

45.90.142.58
Tokyo
185.220.100.245

185.220.101.38

209.141.60.19
Las Vegas
171.25.193.78

185.56.80.65

194.48.199.78

213.95.149.22
Arberg
204.8.156.142
Boston
194.48.199.78

194.48.199.78

185.220.101.32

45.90.142.58
Tokyo
194.48.199.78

171.25.193.20

45.90.142.58
Tokyo
45.90.142.58
Tokyo
107.189.1.178
Roost
185.220.101.160

185.220.100.251

185.220.100.254

185.220.100.255

185.38.175.132

185.220.101.136

194.48.199.78

185.220.101.180

185.220.101.180

185.220.100.253

45.90.142.58
Tokyo
185.220.100.253

185.220.101.146

45.90.142.58
Tokyo
185.220.101.147

194.48.199.78

194.48.199.78



ポート番号を見てみる

In [26]:
from DataGrouper import groupByPorts
r_ports=groupByPorts(r1)
r_ports=sorted(r_ports.items(),key=lambda x:len(x[1]),reverse=True)
print(len(r_ports))
for port, g in r_ports:
    print("%s : %d" % (port,len(g)))

1047
80 : 998
8080 : 928
8983 : 391
102 : 211
7547 : 190
8089 : 177
8085 : 167
8081 : 154
81 : 153
9200 : 144
8000 : 141
8088 : 99
8087 : 83
8086 : 82
8084 : 72
5984 : 65
8888 : 61
9092 : 59
8010 : 55
3689 : 52
8008 : 50
16992 : 48
1400 : 47
8834 : 47
8083 : 46
7548 : 43
7001 : 41
18080 : 40
8500 : 38
50880 : 37
7100 : 37
4040 : 36
443 : 35
5000 : 35
4567 : 34
1024 : 33
8001 : 32
8181 : 31
7077 : 31
49152 : 31
1194 : 29
9002 : 29
7549 : 29
8899 : 28
3128 : 28
1433 : 27
50100 : 27
13579 : 27
9001 : 27
6060 : 27
49153 : 26
7000 : 26
3000 : 25
5555 : 24
9990 : 24
8545 : 24
2004 : 24
4001 : 24
7010 : 24
8090 : 23
27017 : 22
5431 : 22
9060 : 22
8820 : 22
65535 : 22
8761 : 22
10000 : 21
110 : 21
60000 : 21
995 : 20
992 : 19
9090 : 19
666 : 19
7011 : 19
8060 : 19
554 : 18
9000 : 18
7678 : 18
88 : 18
21 : 17
5901 : 16
1026 : 16
9944 : 16
6066 : 16
8880 : 15
1311 : 15
22 : 15
9870 : 15
664 : 15
7002 : 14
4000 : 14
1521 : 13
5005 : 13
16993 : 13
4443 : 13
623 : 13
8554 : 12
50021 : 12
3390 : 12
